# Linear and Logistic Regression

* Matteo Facchetti
* Mario Damiano Russo
* Mirko Frigerio

Here you can find an example of how to perform Linear and Logistic Regression with Python. I will present a few exercises and the solution that my team has come up with.

For these exercises, please refer to the dataset `wines_properties.csv` inside the Data folder.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exercise-1---Linear-Regression" data-toc-modified-id="Exercise-1---Linear-Regression-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exercise 1 - Linear Regression</a></span></li><li><span><a href="#Exercise-2---Logistical-Regression" data-toc-modified-id="Exercise-2---Logistical-Regression-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Exercise 2 - Logistical Regression</a></span></li><li><span><a href="#Exercise-3---Principal-Component-Analysis" data-toc-modified-id="Exercise-3---Principal-Component-Analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exercise 3 - Principal Component Analysis</a></span><ul class="toc-item"><li><span><a href="#Linear-regression" data-toc-modified-id="Linear-regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Linear regression</a></span></li><li><span><a href="#Logistic-regression" data-toc-modified-id="Logistic-regression-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Logistic regression</a></span></li></ul></li><li><span><a href="#Exercise-4---Comments" data-toc-modified-id="Exercise-4---Comments-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Exercise 4 - Comments</a></span></li></ul></div>

In [8]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [9]:
data = pd.read_csv("wines_properties.csv")
data.describe()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,1.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,1.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,1.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,2.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,3.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,3.000000


## Exercise 1 - Linear Regression

Implement a linear regression to predict the percentage of alcohol of the wine, using the other variables.

`Customer_Segment` is considered as a numerical variable but we need to _encode_ it and consider it as a categorical variable.

In [10]:
dummy_segment = pd.get_dummies(data["Customer_Segment"], prefix = "segment")
dummy_segment.head()

,segment_1,segment_2,segment_3
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


Now we have to join this dataframe with the original one and drop one of the columns for multicollinearity reasons.

In [11]:
df = data[ ['Alcohol', 'Malic_Acid', 'Ash', 'Ash_Alcanity', 'Magnesium',
       'Total_Phenols', 'Flavanoids', 'Nonflavanoid_Phenols',
       'Proanthocyanins', 'Color_Intensity', 'Hue', 'OD280', 'Proline'] ]\
.join(dummy_segment.loc[:, "segment_2" :])
df.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,segment_2,segment_3
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,0,0
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,0,0
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,0,0
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,0,0
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,0,0


We can now perform the regression. We first need to add a column of ones in order to find the intercept of our regression. Then we can fit and summarize the OLS model.

In [12]:
# Intercept
df = sm.add_constant(df, prepend = False)

# Fit and summarize the OLS model
mod = sm.OLS(endog = df["Alcohol"], exog = df[df.columns[1:]])
res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                Alcohol   R-squared:                       0.663
Model:                            OLS   Adj. R-squared:                  0.634
Method:                 Least Squares   F-statistic:                     22.89
Date:                Fri, 28 Dec 2018   Prob (F-statistic):           1.52e-31
Time:                        22:55:18   Log-Likelihood:                -118.20
No. Observations:                 178   AIC:                             266.4
Df Residuals:                     163   BIC:                             314.1
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Malic_Acid               0.0745 

## Exercise 2 - Logistical Regression

Implement a logistical regression to predict if a wine is "strong" or "weak".

In [25]:
df.describe()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,segment_2,segment_3,const
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.0
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.398876,0.269663,1.0
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.491049,0.445037,0.0
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000,0.000000,1.0
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000,0.000000,1.0
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,0.000000,0.000000,1.0
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,1.000000,1.000000,1.0
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,1.000000,1.000000,1.0


We need to create a dummy variable using a threshold to discretize the alcohol variable. We consider a wine to be strong if its percentage of alcohol is above the 75$^{th}$ percentile (13.6775), but the threshold is arbitrary and other ways can be found.

In [14]:
df["Strong"] = 1
df.loc[df.Alcohol < 13.6775, "Strong"] = 0

Now we can set up and fit the model.

In [35]:
# Setting the model
logistical_regression = sm.Logit(df["Strong"], df[df.columns[1 : -1]])

# Fitting the model
fitted_model = logistical_regression.fit()

fitted_model.summary2()

Optimization terminated successfully.
         Current function value: 0.316661
         Iterations 9


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
======================================================================
Model:                 Logit             Pseudo R-squared:  0.440     
Dependent Variable:    Strong            AIC:               142.7312  
Date:                  2018-12-07 18:49  BIC:               190.4580  
No. Observations:      178               Log-Likelihood:    -56.366   
Df Model:              14                LL-Null:           -100.64   
Df Residuals:          163               LLR p-value:       7.1213e-13
Converged:             1.0000            Scale:             1.0000    
No. Iterations:        9.0000                                         
----------------------------------------------------------------------
                      Coef.   Std.Err.    z    P>|z|   [0.025   0.975]
----------------------------------------------------------------------
Malic_Acid             0.5382   0.3159  1.7035 0.0885  -0.0810  1.1574
Ash                   -0.6709   1.5298 -0.4385 0.6610  -3.6692  2.3275
Ash_Alcanity          -0.0454   0.1269 -0.3573 0.7209  -0.2942  0.2034
Magnesium             -0.0205   0.0245 -0.8364 0.4029  -0.0686  0.0276
Total_Phenols          1.1029   1.0512  1.0491 0.2941  -0.9575  3.1632
Flavanoids             0.4805   0.9785  0.4911 0.6234  -1.4374  2.3984
Nonflavanoid_Phenols   2.2341   3.3196  0.6730 0.5009  -4.2722  8.7404
Proanthocyanins       -0.3791   0.7684 -0.4934 0.6217  -1.8852  1.1269
Color_Intensity        0.5632   0.2491  2.2609 0.0238   0.0750  1.0514
Hue                    5.6099   2.4720  2.2694 0.0232   0.7649 10.4549
OD280                  1.4103   0.8291  1.7010 0.0889  -0.2147  3.0354
Proline                0.0010   0.0016  0.6241 0.5325  -0.0021  0.0041
segment_2             -2.9181   1.6316 -1.7885 0.0737  -6.1159  0.2798
segment_3              2.1586   2.5465  0.8477 0.3966  -2.8324  7.1496
const                -15.0776   5.4588 -2.7620 0.0057 -25.7767 -4.3784
======================================================================

"""

## Exercise 3 - Principal Component Analysis

Implement linear and logistic regression using as regressors the first principal components after the PCA.

Let's perform the PCA first. You can find other exercises about Factor Analysis and Cluster Analysis performed on this dataset [here](https://github.com/MatteoFacchetti/StatisticsExercises/blob/master/FactorClusterAnalysis.ipynb).

To perform the PCA we first standardize our data, then we get the eigenvalues and the eigenvectors of the correlation matrix and we sort them in order to see what is the variance explained by the Principal Components. The PCA cannot be performed with categorical variables, therefore we will get rid of the _Customer Segment_ variable and we will add it back to the dataset afterwards.

In [115]:
# Standardization
data_s = StandardScaler().fit_transform(data.iloc[:, 1:13])

# Get eigenvalues and eigenvectors
correlation_matrix        = np.corrcoef(data_s.T)
eigenvalues, eigenvectors = np.linalg.eig(correlation_matrix)

# Ratio of explained variance
tot_eigenvalues    = sum(eigenvalues)
sorted_eigenvalues = sorted(eigenvalues, reverse = True)
variance_explained = [(i / tot_eigenvalues) * 100 for i in sorted_eigenvalues]
np.cumsum(variance_explained)

array([ 38.61231788,  55.49347493,  66.87003122,  74.52677438,
        81.16309995,  86.24109272,  90.81376922,  93.54766395,
        95.68703488,  97.62432816,  99.13806849, 100.        ])

The first 4 components explain almost 75% of the variance. We will use them as regressors.

In [116]:
eigen = [ (np.abs(eigenvalues[i]), eigenvectors[:, i]) for i in range(len(eigenvalues))]

# Sort the eigenvalues
eigen.sort()

# Now sort in reverse order
eigen.reverse()

# Creating the top-4 eigenvectors matrix
top4 = np.hstack( (eigen[0][1].reshape(12, -1),
                  eigen[1][1].reshape(12, -1),
                  eigen[2][1].reshape(12, -1),
                  eigen[3][1].reshape(12, -1)) )

# Get the new observations
Y = pd.DataFrame(data_s.dot(top4))
Y.columns = ["PC{}".format(i) for i in range(1, 5)]
Y = Y.join(data["Alcohol"]).join(dummy_segment.loc[:, "segment_2" :])

# Move Alcohol to first position
cols = Y.columns.tolist()
cols = cols[4:5]+cols[:4]+cols[5:]
Y    = Y[cols]
Y.head()

,Alcohol,PC1,PC2,PC3,PC4,segment_2,segment_3
0,14.23,3.078573,-1.219262,0.388070,-0.239523,0,0
1,13.20,2.190796,0.664568,1.922109,-0.291411,0,0
2,13.16,2.456441,-1.505811,-0.556489,0.723596,0,0
3,14.37,3.422750,-2.556058,0.703346,0.564575,0,0
4,13.24,0.948976,-1.329735,-1.760632,-0.411977,0,0


### Linear regression

Now we can perform the linear regression of _Alcohol_ on the principal components using the same logic as before.

In [117]:
# Intercept
Y = sm.add_constant(Y, prepend = False)

# Fit and summarize the OLS model
mod = sm.OLS(endog = Y["Alcohol"], exog = Y[Y.columns[1:]])
res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                Alcohol   R-squared:                       0.630
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     48.48
Date:                Sat, 08 Dec 2018   Prob (F-statistic):           1.94e-34
Time:                        02:19:17   Log-Likelihood:                -126.53
No. Observations:                 178   AIC:                             267.1
Df Residuals:                     171   BIC:                             289.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
PC1            0.0028      0.042      0.067      0.9

### Logistic regression

Finally, let's perform the logit regression.

In [118]:
Y["Strong"] = 1
Y.loc[Y.Alcohol < 13.6775, "Strong"] = 0

# Setting the model
logistical_regression = sm.Logit(Y["Strong"], Y[Y.columns[1 : -1]])

# Fitting the model
fitted_model = logistical_regression.fit()

fitted_model.summary2()

Optimization terminated successfully.
         Current function value: 0.357431
         Iterations 8


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.368     
Dependent Variable: Strong           AIC:              141.2455  
Date:               2018-12-08 02:19 BIC:              163.5180  
No. Observations:   178              Log-Likelihood:   -63.623   
Df Model:           6                LL-Null:          -100.64   
Df Residuals:       171              LLR p-value:      6.0582e-14
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     8.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
PC1            0.6602    0.3544   1.8630  0.0625  -0.0344   1.3548
PC2           -0.3270    0.2795  -1.1701  0.2420  -0.8748   0.2207
PC3            0.0402    0.2695   0.1491  0.8815  -0.4881   0.5685
PC4            0.2415    0.2837   0.8513  0.3946  -0.3146   0.7977
segment_2     -3.2495    1.1816  -2.7501  0.0060  -5.5655  -0.9336
segment_3      1.2284    1.8059   0.6802  0.4964  -2.3112   4.7679
const         -1.2356    0.7149  -1.7284  0.0839  -2.6367   0.1655
=================================================================

"""

## Exercise 4 - Comments

Provide some indication about the overall fitting for the first problem (as R-squared) and about the accuracy of the prediction for the second problem. Compare the results you get with the two approaches (non-PCA vs PCA).

Both the first problem ($R^2=0.663$) and the second problem ($pseudo\;R^2=0.440$) seem to be good models to explain the variability of our dependent variable. Of course when we perform the PCA, the R-squared decreases, because we give up a little amount of variability in order to simplify our model and make it easier to interpret, but the decrease is small enough to be negligible.